#### (Szymon Talaga, 11.11.2019) | Parts adapted from Julian Zubek 11.2018
<hr>

## Jupter notebooks & Jupyter lab

* It originated from the IPython project.
* Jupyter = **Ju**lia + **Py**thon + **R** (currently it supports even more languages).
* It supports *literate programming*.
* Quite handy and useful for interactive data analsis, sharing and presentation of results and so on.
* One of the most popular tools of this type in the data science community.

Launching a notebook:

```bash
jupyter notebook lab_02.ipynb
```

Jupyter lab is a new improved version of the Jupyter notebook environment. It supports working with and viewing multiple notebooks, docking visualization results to separate windows and linking notebooks with Python shells, what can make interactive work even more seamless.

In this class we will be using Jupyter lab or Jupyter notebook environments.

Launching a lab:

```bash
jupyter lab
# Should be run in the folder where notebooks reside
```

# Advanced iteration utilities and data structures
## Iterators, generators and packages `itertools` and `collections`

Now, after the initial warm up, we will introduce ourselves to some core Python tools available in the so-called standard library (a set of Python packages distributed with Python by default) which are very useful in general data processing.

`itertools` module contain a lot of very useful utilities for iterating and transforming data on the fly. The core principle of `itertools` is memory-independence, which means that all provided procedures can iterate over arbitrarily large objects without ever needing to store them in memory. This can be done thanks to **generators** and **iterators**.

Both of them are special kinds of objects that iterate (and possibly do something else) over an iterable collection of objects in such a manner that at any point in time only one object is stored in memory. Let us see this with an example.

In [1]:
range(10) # Does not return a list of numbers

# This is so, because it is an iterator and at every point in time
# only the current number is stored in the memory.
#
# This can be very useful when do things like:
for x in range(1000000):
    pass # do something here

# If we treated range like a normal list, instead of an iterator
# we would have first to initialize the whole list of 1000000 numbers
# in the memory and only after that iterate over them.

range(0, 10)

Ok, but what is the difference between iterators and generators? They both allow a single pass over an iterable collection that was passed to them. What does that mean?

In [2]:
iterator = iter(range(10))
[ x for x in iterator ]
[ x for x in iterator ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[]

The difference between iterators and generators is quite technical. The concept of an iterator is older and to create an iterator we have to define a class implementing a specific type of a protocol (a class that implements special methods `__iter__` and `__next__`), called iteration protocol. Or we can use the built-in `iter` function (as above) to create an iterator based on some iterable collection, but this will work only for simple usecases.

Generators on the other hand are younger and easier to use, because we can implement an arbitrarily complex generator just through a simple function. To do so we use a special `yield` keyword. They are a subclass of iterators, so every generator is an iterator, but not *vice versa*.

In [3]:
def range_generator(n):
    i = 0
    while i < n:
        yield i
        i += 1

[ x for x in range_generator(10) ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

So it seems we just implemented our own version of the built-in range function! Cool, huh? Not that fast. We mentioned that that iterators and generators allow for only one pass. After that they become empty (this is not a bug, but a feature, since this behavior can be sometimes incredibly useful). So let us check what happens when we pass over our range generator multiple times.

In [4]:
r = range(10)
r_gen = range_generator(10)

# Pass two times over the generator
[ x for x in r_gen ]
[ x for x in r_gen ]

# Pass to times over the standard range
[ x for x in r ]
[ x for x in r ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

Why built-in range allow for multiple passes? Does this mean that under the hood it is not as memory friendly as our range generator? No, it still stores only one number at any given point in time. But what it does under the hood is it returns a new iterator every time we try to iterate over it. So it does not break the one pass rule. This way, the standard `range` object is a **memory-independent** iterable.

Take home message is that we have the following basic distinction:

* **iterators/generators:** allow one pass, but are memory-independent by design (although if you really want you can implement a generator that is not memory-independent).
* **iterables:** allow multiple passes; can be either memory-independent or not, but if they are, they will always internally use iterators/generators. 

In [5]:
# __iter__ is a magic method that is called everytime we try to iterate over something
r_iteration = r.__iter__()

[ x for x in r_iteration ]
[ x for x in r_iteration ]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

[]

How can we differentiate between iterators/generators and iterables? One way is to use the built-in `next` function. Iterables do not support `next` statements as they do not have the internal state that tells tham where they are in the loop at the current moment. At the other hand iterators and generators have internal state and support `next` statements.

In [6]:
gen_rng = range_generator(5)
next(gen_rng)
next(gen_rng)
next(gen_rng)
next(gen_rng)
next(gen_rng)
next(gen_rng)

0

1

2

3

4

StopIteration: 

Nice thing about generators is that we can define simple generators as generator comprehensions in the same way as we can write list comprehensions.

In [7]:
## List comprehension
[ x*2 for x in range(5) ]

## Generator comprehension
(x*2 for x in range(5))

[0, 2, 4, 6, 8]

<generator object <genexpr> at 0x7f37e5f78b10>

Why is this useful? For instance because generator expressions allows us to pass data to accumulating functions without having to store all the data in memory.

In [8]:
list_sum = sum([ x**2 for x in range(1000) ])
gen_sum = sum(x**2 for x in range(1000))

assert list_sum == gen_sum

## The results are the same, but since the second sum was defined
## via a generator (and range itself is an iterable generator)
## during the entire summation process it stored only two numbers
## at any point in time: the current sum and a new number to add to it.

### Built-in Python iterators

Python provides several very useful built-in iterators.

In [9]:
## ZIP: align multiple collections together
for x, y, z in zip([1,2,3], [4,5,6], [7,8,9]):
    print(x, y, z)

1 4 7
2 5 8
3 6 9


In [10]:
## But beware that ZIP limits itself to the shortest collection
## without letting us know.
for x, y in zip([1], [2,3,4,5,6]):
    print(x, y)

1 2


In [11]:
## Map: apply a function to each element
for x in map(lambda x: x**2, [1,2,3]):
    print(x)

1
4
9


In [12]:
## Filter: remove elements that do not pass a test
for x in filter(lambda x: x % 2 == 0, [1,2,3,4]):
    print(x)

2
4


In [13]:
## Iterate over numbered elements
for i, x in enumerate(['a', 'b', 'c', 'd']):
    print(i, x)

0 a
1 b
2 c
3 d


In [14]:
## This equivalent to, but much more readable than:
collection = ['a', 'b', 'c', 'd']
for i, x, in zip(range(len(collection)), collection):
    print(i, x)

0 a
1 b
2 c
3 d


In [15]:
## Sorted: iterate over sorted collection (if it is sortable)
for x in sorted([2, 7, 1, 3, 0, 11, 4, 1]):
    print(x)

0
1
1
2
3
4
7
11


In [16]:
## Reversed: reverse an iterable collection
for x in reversed([1, 2, 3]):
    print(x)

3
2
1


### `itertools`

The core design principle of the `itertools` package is that every function returns an iterator. This means that we have **ONLY ONE PASS** over objects returned by `itertools` functions.

The package defines quite a lot of useful functions and we have time to get accquainted with only few of them. You can easily learn more from the `itertools` [documentation](https://docs.python.org/3/library/itertools.html).

In [17]:
import itertools as it

## We can use `cycle` to cycle indefinitely over a collection
infinite_cycle = it.cycle([1, 2, 3])

for _ in range(10):
    print(next(infinite_cycle))

1
2
3
1
2
3
1
2
3
1


In [18]:
## We can repeat something multiple times
list(it.repeat('something', 5))

['something', 'something', 'something', 'something', 'something']

In [19]:
## Or chain multiple iterables into one long iterable
list(it.chain([1, 2], [3, 4]))

[1, 2] + [3, 4]

[1, 2, 3, 4]

[1, 2, 3, 4]

Quite often we want to get a cartesian product of multiple collections (all possible combinations of their elements).
This is very useful for instance if we want to run some code multiple times with different combinations of arguments,
for instance when we run a simulation study.

In [20]:
letters = ['a', 'b', 'c']
numbers = [1, 2]

for letter, number, x in it.product(letters, numbers, ['c', 'd']):
    print(letter, number, x)

a 1 c
a 1 d
a 2 c
a 2 d
b 1 c
b 1 d
b 2 c
b 2 d
c 1 c
c 1 d
c 2 c
c 2 d


In [21]:
# EXERCISE 1.
#
# Write a list comprehension expression using iterator utilities
# such as map, filter or product to produce a multiplication
# table for all numbers from 1 to 10.
#
# The result should be a list of 10 lists of 10 elements
# (100 elements in total) matching the `expected` list below.
#
# HINT: remember that iterators does not return anything
# before they are iterated over. To dump an iterator
# to a list you can call the `list` constructor method on it.
#
# Like this:
#       range(3) != [0, 1, 2]
# list(range(3)) == [0, 1, 2]

numbers = list(range(1, 11))

# VERSION 1.
result = [ [ i*j for j in numbers] for i in numbers ]
# VERSION 2.
# It is more convoluted, but use a `lambda` expression in an interesting way,
# since part of the logic in the expression (the `i` variable)
# comes from the outer for-loop.
result = [ list(map(lambda x: x*i, numbers)) for i in numbers ]

expected = [
    [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
    [3, 6, 9, 12, 15, 18, 21, 24, 27, 30],
    [4, 8, 12, 16, 20, 24, 28, 32, 36, 40],
    [5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    [6, 12, 18, 24, 30, 36, 42, 48, 54, 60],
    [7, 14, 21, 28, 35, 42, 49, 56, 63, 70],
    [8, 16, 24, 32, 40, 48, 56, 64, 72, 80],
    [9, 18, 27, 36, 45, 54, 63, 72, 81, 90],
    [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
]

assert result == expected, "Sorry, your answer is not correct."

In [22]:
# EXERCISE 2:
#
# You are running a simulation study in which you want to explore a parameter space
# Defined by 2 discrete parameters `alpha` and `beta`.
# You want to run your simulation for all possible combinations of values
# and run 5 independent simulations for each combination.
#
# This time you do not want to store everything in memory, but
# rather run over parameters' combinations one by one,
# so your expression should return an iterator/generator.
#
# Identical combinations should be grouped together (appear one after another)
# but the iterator itself has to be flat.
#
# You can use iterator utilities such as map, product, chain or repeat,
# but you can also write your own generator with `yield` statements if you will.
#
# HINT: remeber that you can flatten list of arguments with a star `*`
#
# Like this:
#
# def add(x, y): return x + y
#
# two_numbers = (1, 2)
#
# add(*two_numbers) == 3

n_repeats = 5   
alpha     = [1, 2, 3]
beta      = [10, 20]


# VERSION 1.
result = it.chain(*(list(it.repeat(params, n_repeats)) for params in it.product(alpha, beta)))
# VERSION 2. (with chain.from_iterable)
result = it.chain.from_iterable(list(it.repeat(params, n_repeats)) for params in it.product(alpha, beta))

expected = [
    (1, 10), (1, 10), (1, 10), (1, 10), (1, 10),
    (1, 20), (1, 20), (1, 20), (1, 20), (1, 20),
    (2, 10), (2, 10), (2, 10), (2, 10), (2, 10),
    (2, 20), (2, 20), (2, 20), (2, 20), (2, 20),
    (3, 10), (3, 10), (3, 10), (3, 10), (3, 10),
    (3, 20), (3, 20), (3, 20), (3, 20), (3, 20)
]

assert list(result) == expected, "Sorry, your answer is not correct."

`itertools` package allows us also to define some common data processing and statistical operation as memory-independent iterators.

In [23]:
import itertools as it

## Running sum
list(
    it.accumulate([1, 2, 3])
)

[1, 3, 6]

In [24]:
## Running product

list(
    it.accumulate([2, 2, 2], lambda x, y: x*y)
)

[2, 4, 8]

In [25]:
## Remove elements until a condition is met
list(
    it.dropwhile(lambda x: x < 5, [ 1, 2, 3, 7, 8, 1, 1, 2])
)

[7, 8, 1, 1, 2]

In [26]:
## Take elements until a condition is met
list(
    it.takewhile(lambda x: x < 5, [ 1, 2, 3, 7, 8, 1, 1, 2])
)

[1, 2, 3]

In [27]:
## Nifty trick (in base Python) to take a first element that meets a conditon
## It is based on a generator-comprehension
gen = (x for x in range(1, 10001) if (x % 47 == 0) and (x % 11) == 0)

In [28]:
## Filter elements by a list of logical values (or interpretable as booleans)
list(
    it.compress([1, 2, 3, 4, 5], [0, 0, 1, 0, 1])
)

[3, 5]

In [29]:
numbers = [1, 2, 3, 4]

mapping = {
    0: [2, 4],
    1: [1, 3]
}
mapping

{0: [2, 4], 1: [1, 3]}

In [30]:
L = [3, 5, 4, 7, 1, 0, 11, 9, 6]
list(sorted(L, key=lambda x: x % 2))

[4, 0, 6, 3, 5, 7, 1, 11, 9]

In [31]:
## Group by: returned values grouped by a key function
## It is a complicated function, so it is worthwhile to read its documentation:
##     https://docs.python.org/3/library/itertools.html#itertools.groupby
##
## The key function when called on elements of an iterable has to return
## an ordinal object and the iterable has to be already sorted by the key
## function.
##
## Let us look at an example.
##
## We will try to group a list of numbers into tens, like 0-9, 10-19, etc.

## integer division by 10
keyfunc = lambda x: x // 10
## Sorted function accepts additional argument which is a sort key function
numbers = sorted(range(100), key=keyfunc)
{ k: list(g) for k, g in it.groupby(numbers, keyfunc) }

{0: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 1: [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 2: [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
 3: [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
 4: [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
 5: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
 6: [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
 7: [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
 8: [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
 9: [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]}

In [32]:
# EXERCISE 3.
#
# Write using whatever means necessary (though we recommend using itertools functions) 
# A function called `grouped_data` that will organize the data below (exam scores for students)
# in the following way:
#
# Students have to be grouped by the first letter of the name (uppercase)
# and sorted by scores within groups.
# In every group running sum score has to be computed.
# But students with negative scores (did not take the exam) have to be filtered out first.
# Every group has to be represented as a dict with two keys:
#     1. students: list of students who took the exame
#     2. run_score: running sum of exam score.
#
# HINT: look closely at the expected output.

data = [
    ('Bob', 7), ('Alice', 8), ('Marcin', 11), ('Julia', 13), ('Adam', 3),
    ('Zenon', 11), ('Zygmunt', -1), ('Tiago', 2), ('Jagna', 0), ('Joda', -1),
    ('Luke', 1), ('Chewbaca', -1), ('Agata', 2), ('Sara', 10), ('Darth Vader', 5)
]

def grouped_data(data):
    keyfunc = lambda x: x[0][0].upper()
    data = (x for x in data if x[1] >= 0)
    data = sorted(data, key=keyfunc)
    dct = {}
    for k, g in it.groupby(data, keyfunc):
        #print("k = ", k, "; g =", list(g))
        g = list(sorted(g, key=lambda x: x[1]))
        students = [ name for name, score in g ]
        scores = [ score for name, score in g ]
        group_data = {
            'students': students,
            'run_score': list(it.accumulate(scores))
        }
        dct[k] = group_data
    return dct

expected = {
    'A': {'students': ['Agata', 'Adam', 'Alice'], 'run_score': [2, 5, 13]},
    'B': {'students': ['Bob'], 'run_score': [7]},
    'D': {'students': ['Darth Vader'], 'run_score': [5]},
    'J': {'students': ['Jagna', 'Julia'], 'run_score': [0, 13]},
    'L': {'students': ['Luke'], 'run_score': [1]},
    'M': {'students': ['Marcin'], 'run_score': [11]},
    'S': {'students': ['Sara'], 'run_score': [10]},
    'T': {'students': ['Tiago'], 'run_score': [2]},
    'Z': {'students': ['Zenon'], 'run_score': [11]}
}

assert grouped_data(data) == expected, "Sorry, your answer is not correct."""

## More warm up exercises

1\. Write an expression that reverses a string (e.g. "abc" -> "cba").

In [33]:
string = "abc"

# VERSION 1.
string[::-1]

# VERSION 2.
"".join(reversed(string))

'cba'

'cba'

2\. Write a function that takes a string and returns `True` if the string is a palindrom (reads the same from the left the right) and `False` otherwise. Can you reuse the previous result?

In [34]:
def is_palindrome(string):
    return string == "".join(reversed(string)) and len(string) > 1 and " " not in string

3\. Write a function that takes a string and returns all the palindrome substring (they do not have to be meaningful words, but should be longer than one character and not contain any whitespaces). Can you reuse your previous results? And can you write this as a one-liner comprehension?

In [35]:
## Test on this string
s = "This summer our long neglected civic duty brought the kayak back on our radar."

In [36]:
def get_palindromes(s):
    for i in range(len(s)):
        for j in range(i+1, len(s)):
            if is_palindrome(s[i:j]):
                yield s[i:j]

4\. Write a one liner expression that flattens a list of list to a single-level list.

In [37]:
L = [ [1,2,3], [2,3], [1], [], [10,1010,1]] # Should be [1,2,3,2,3,1,10,1010,1]

# SOLUTION.
[ number for sublist in L for number in sublist ]

# IT IS EQUIVALENT TO
# BUT MUCH MORE EFFICIENT FROM:
flat_L = []
for sublist in L:
    for number in sublist:
        flat_L.append(number)
flat_L

[1, 2, 3, 2, 3, 1, 10, 1010, 1]

[1, 2, 3, 2, 3, 1, 10, 1010, 1]

5\. Write a a function that takes an integer `n` as input and generate an `n`-level dollar sign christmas tree like the one below. Can you write it as a one liner?

```
  $ 
 $$$
$$$$$
```

In [39]:
def print_pyramid(n):
    for i in range(n):
        print(" "*(n-i-1) + "$"*(2*i+1))

print_pyramid(10)

         $
        $$$
       $$$$$
      $$$$$$$
     $$$$$$$$$
    $$$$$$$$$$$
   $$$$$$$$$$$$$
  $$$$$$$$$$$$$$$
 $$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$


In [40]:
## BELOW I MODIFY THE FUNCTION TO MAKE IT RETURN A STRING.
def pyramid_string(n):
    return "\n".join(" "*(n-i-1) + "$"*(2*i+1) for i in range(n))

print(pyramid_string(10))

         $
        $$$
       $$$$$
      $$$$$$$
     $$$$$$$$$
    $$$$$$$$$$$
   $$$$$$$$$$$$$
  $$$$$$$$$$$$$$$
 $$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$$$$$$$


## Recurrence

Recurrsive function are functions that call themselves. In order not to end up stuck in an infinite loop they have to do this in a smart way, so at every call they move towards some well-defined stopping criterion. Let us what that means based on arguably the most famous recurrsive formula of all times: the Fibonnaci sequence.

$$x_1 = 1$$
$$x_2 = 1$$
$$x_n = x_{n-2} + x_{n-1}$$

It gives the following sequence (it is infinite so we show ony the beginning):
$$
1, 1, 2, 3, 5, 8, 13, 21 \ldots
$$

Can you code a function that given a positive integer `n` returns the `n`-th Fibonnaci number? Do this in a recursive (function has to call itself) way that mimics the mathematical definition as close as possible.

In [41]:
def fibonnaci(n):
    if n <= 2:
        return 1
    return fibonnaci(n-1) + fibonnaci(n-2)

Now let us see how well does this function behaves for large n.

In [42]:
%%time
fibonnaci(38)

CPU times: user 6.97 s, sys: 0 ns, total: 6.97 s
Wall time: 6.98 s


39088169

Are you happy with the execution time? No, I though so, you should not be. Something is off here. Can you guess what it is? How could reimplement the function to make it fast?

In [49]:
def fast_fibbonaci(n):
    n0 = n1 = 1
    if n <= 2:
       return 1
    for i in range(n-2):
        n0, n1 = n1, n0 + n1
    return n1

In [44]:
%%time
## COMPARE TIMES
##
## SLOW FIBONNACI
fibonnaci(40)

CPU times: user 17.8 s, sys: 459 µs, total: 17.8 s
Wall time: 17.8 s


102334155

In [51]:
%%time
## COMPARE TIMES
##
## FAST FIBONNACI
fast_fibbonaci(400)

CPU times: user 29 µs, sys: 0 ns, total: 29 µs
Wall time: 31.5 µs


176023680645013966468226945392411250770384383304492191886725992896575345044216019675

Ok, we solved the problem. So it seems that recursion, as beutiful mathematically it may be, is not really useful in computer programming, because it makes things slow, right? Not exactly. This was just an example of a slow recurssion that should be avoided.

Let us now look at a very nice example of recursive algorithm which really can not be expressed differently in the general case.

In [53]:
# EXCERCISE 4.
#
# Write a function that flattens a list of nested lists of nested lists etc.
# with arbitrary depth of nesting.
#
# Implement it as a memory-independent generator.
# This way you will not have to copy the list to flatten it.

nested_L = [
    [1, [2, 3, []], [1, 2, 3, [1, [2, 3, []]]]],
    [1, 2, 3, [2, 3, [], [], [3,4]]]
]

def flatten(L):
    for element in L:
        if isinstance(element, list):
            for subelement in flatten(element):
                yield subelement
            # Shorthand for this is:
            # yield from flatten(element)
        else:
            yield element

# Solution based on dynamically appending to a list.
# Work, but it is a bad implementation.
# It is slower and a little bit mor difficult to read and write.
def flatten2(L):
    flat_L = []
    for element in L:
        if isinstance(element, list):
            flat_L += flatten2(element)
        else:
            flat_L.append(element)
    return flat_L
            

expected = [1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 2, 3, 3, 4]
assert list(flatten(nested_L)) == expected, "Sorry, your answer is not correct."
assert list(flatten2(nested_L)) == expected, "Sorry, your answer is not correct."

## Package `collections`

Since we are already quite well versed with iterations and traversing data (also recursively) it is time to look at the `collections` package which provides us with few very useful data structures.

### Ordered dict

`OrderedDict` is just an ordinary `dict` but such that keeps the order of key-value pairs. Normal dictionaries do not care about the order and may arrange key-value pairs randomly.

In [55]:
from collections import OrderedDict

# This is a BAD way of initializing an ordered dict. Why?
# Because the standard dictionary we initialize first already
# messes up our ordering, so the final ordered dict will
# have random order anyway.
dct = {'a': 1, 'b': 2, 'c': 3}
odict = OrderedDict(dct)
odict

OrderedDict([('a', 1), ('b', 2), ('c', 3)])

In [56]:
from collections import OrderedDict

odict = OrderedDict([
    ('z', 2),
    ('c', 1),
    ('a', 3),
    ('b', 4)
])
# REMEMBER. ONLY BY DEFINING THROUGH A LIST OF TUPLES (OR LIST OF LISTS)
# YOU CAN REALLY ENFORCING A PARTICULAR ORDERING.
#
# CONVERTING A DICT INTO AN ORDERED DICT WILL NOT DO,
# AS ORDINARY DICTS HAVE RANDOM ORDERING.

for k, v in odict.items():
    print(k, v)

z 2
c 1
a 3
b 4


### Default dict

This is one of my favourites. It is simple yet in some circumstances very useful. This is a dictionary that have a default value that it returns when asked for a key it does not have. And once asked, it initializes the key with the default value.

It is super useful when you build dynamically a dictionary of lists that themselves are dynamically extended. Or when you build a dict with nested dicts in the same way.

In [59]:
from collections import defaultdict

lorem = \
    "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. " \
    "Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. " \
    "Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. " \
    "Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum." \

## We will build a dict with lists of word beginning with paticular letters.
## It will be a dict with list as a default.
ldict = defaultdict(list)
dct = {}

for word in lorem.split():
    word = word.strip().lower()
    if not word:
        continue
    ldict[word[0].upper()].append(word)

ldict

defaultdict(list,
            {'L': ['lorem', 'labore', 'laboris', 'laborum.'],
             'I': ['ipsum', 'incididunt', 'irure', 'in', 'in', 'in', 'id'],
             'D': ['dolor',
              'do',
              'dolore',
              'duis',
              'dolor',
              'dolore',
              'deserunt'],
             'S': ['sit', 'sed', 'sint', 'sunt'],
             'A': ['amet,',
              'adipiscing',
              'aliqua.',
              'ad',
              'aliquip',
              'aute',
              'anim'],
             'C': ['consectetur',
              'commodo',
              'consequat.',
              'cillum',
              'cupidatat',
              'culpa'],
             'E': ['elit,',
              'eiusmod',
              'et',
              'enim',
              'exercitation',
              'ex',
              'ea',
              'esse',
              'eu',
              'excepteur',
              'est'],
             'T': ['tempor'],
       

That was pretty neat, right? So now solve the same problem but this time use just a standard dict.

In [61]:
## Solve the previous problem with the standard dict.
#
ldict = {}
for word in lorem.split():
    word = word.strip().lower()
    if not word:
        continue
    key = word[0].upper()
    if key in ldict:
        ldict[key].append(word)
    else:
        ldict[key] = [word]

ldict

{'L': ['lorem', 'labore', 'laboris', 'laborum.'],
 'I': ['ipsum', 'incididunt', 'irure', 'in', 'in', 'in', 'id'],
 'D': ['dolor', 'do', 'dolore', 'duis', 'dolor', 'dolore', 'deserunt'],
 'S': ['sit', 'sed', 'sint', 'sunt'],
 'A': ['amet,', 'adipiscing', 'aliqua.', 'ad', 'aliquip', 'aute', 'anim'],
 'C': ['consectetur', 'commodo', 'consequat.', 'cillum', 'cupidatat', 'culpa'],
 'E': ['elit,',
  'eiusmod',
  'et',
  'enim',
  'exercitation',
  'ex',
  'ea',
  'esse',
  'eu',
  'excepteur',
  'est'],
 'T': ['tempor'],
 'U': ['ut', 'ut', 'ullamco', 'ut'],
 'M': ['magna', 'minim', 'mollit'],
 'V': ['veniam,', 'voluptate', 'velit'],
 'Q': ['quis', 'qui'],
 'N': ['nostrud', 'nisi', 'nulla', 'non'],
 'R': ['reprehenderit'],
 'F': ['fugiat'],
 'P': ['pariatur.', 'proident,'],
 'O': ['occaecat', 'officia']}

### Counter

Counter does what it claims it does: it counts occurences of unique objects. It creates a dictionary that maps objects to their numbers of occurences. This means that it can only count hashable (non-mutable) objects such as numbers or tuples but not lists or other dictionaries.

In [62]:
dct = {}
for word in lorem.split():
    word = word.strip().lower()
    if word in dct:
        dct[word] += 1
    else:
        dct[word] = 1

In [63]:
from collections import Counter

words = Counter(word.strip().lower() for word in lorem.split())
words

Counter({'lorem': 1,
         'ipsum': 1,
         'dolor': 2,
         'sit': 1,
         'amet,': 1,
         'consectetur': 1,
         'adipiscing': 1,
         'elit,': 1,
         'sed': 1,
         'do': 1,
         'eiusmod': 1,
         'tempor': 1,
         'incididunt': 1,
         'ut': 3,
         'labore': 1,
         'et': 1,
         'dolore': 2,
         'magna': 1,
         'aliqua.': 1,
         'enim': 1,
         'ad': 1,
         'minim': 1,
         'veniam,': 1,
         'quis': 1,
         'nostrud': 1,
         'exercitation': 1,
         'ullamco': 1,
         'laboris': 1,
         'nisi': 1,
         'aliquip': 1,
         'ex': 1,
         'ea': 1,
         'commodo': 1,
         'consequat.': 1,
         'duis': 1,
         'aute': 1,
         'irure': 1,
         'in': 3,
         'reprehenderit': 1,
         'voluptate': 1,
         'velit': 1,
         'esse': 1,
         'cillum': 1,
         'eu': 1,
         'fugiat': 1,
         'nulla': 1,
       

Do you have any idea how can you sort the counter, so it first are the most frequence items? Remember that it has to remain a mapping.

In [64]:
## Sorted counter (descending order)
odict = OrderedDict((k, v) for k, v in sorted(words.items(), reverse=True, key=lambda x: x[1]))
odict

OrderedDict([('ut', 3),
             ('in', 3),
             ('dolor', 2),
             ('dolore', 2),
             ('lorem', 1),
             ('ipsum', 1),
             ('sit', 1),
             ('amet,', 1),
             ('consectetur', 1),
             ('adipiscing', 1),
             ('elit,', 1),
             ('sed', 1),
             ('do', 1),
             ('eiusmod', 1),
             ('tempor', 1),
             ('incididunt', 1),
             ('labore', 1),
             ('et', 1),
             ('magna', 1),
             ('aliqua.', 1),
             ('enim', 1),
             ('ad', 1),
             ('minim', 1),
             ('veniam,', 1),
             ('quis', 1),
             ('nostrud', 1),
             ('exercitation', 1),
             ('ullamco', 1),
             ('laboris', 1),
             ('nisi', 1),
             ('aliquip', 1),
             ('ex', 1),
             ('ea', 1),
             ('commodo', 1),
             ('consequat.', 1),
             ('duis', 1),
          

### Deque (double-ended queue)

Last but not least we have a super usefull class of `deque`. In some sense it is just like a list (it supports all list operations). However, it enables fast appends and pops from both sides, not only from the right. This can be extremely useful in large, data intensive applciations. Let us look at an example.

In [65]:
from collections import deque

L = list(range(99999))
D = deque(L)

In [66]:
%%time
L.append(0)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [67]:
%%time
D.append(0)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [68]:
%%time
L.insert(0, 1)

CPU times: user 114 µs, sys: 0 ns, total: 114 µs
Wall time: 118 µs


In [69]:
%%time
D.appendleft(1)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.25 µs


In [70]:
%%time
L.pop(0)

CPU times: user 77 µs, sys: 0 ns, total: 77 µs
Wall time: 79.4 µs


1

In [71]:
%%time
D.popleft()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.15 µs


1

We see that for some operations deque can be an order of magnitude faster. Especially, `deque` is the proper class in Python to implement queues, since `list.pop(0, <value>)` becomes slow when data gets big (see the remark on computational complexity below).
However, there is a price to pay. Indexing deques is in general much slower.

## Computational complexity

Estimating exact execution time of programs is hard. However, it is often quite straightforward to assess the worst-case (or average-case) scenario execution time asymptotically, that is assuming very large input data. The assymptotic behavior of programs with respect to execution time is usually described with the so-called **big O** notation.

Most of the basic operations have time complexity somewhere around the following categories:

1. $O(1)$ : means that the execution time does not depend on the data size at all. This is for instance the complexity of key lookup in dictionary or left and right appends in a deque as well as list indexing.
2. $O(\log{}n)$ : execution time scales linearly with respect to the logarithm of the number of elements. This is for instance complexity of binary search.
3. $O(n)$ : execution time grows propotionally to the data size. This is for instance the complexity of a sum over a list of numbers.
4. $O(n\log{}n)$ : this is a typical complexity of something that is just a little bit superlinear. For instance, the Fast Fourier Transform is $O(n\log{}n)$.
5. $O(n^2)$ : Quadratic complexity. If you increase the number of data items two times, then the execution time grows four times. This is bad, we should avoid this whenever we can. A typical scenario that leads to $O(n^2)$ complexity is nesting two loops over the same collection of items. If you write some code and think that two nested loops are a good idea, then think again. Sometimes you can not avoid this of course, but you should always try.
6. $O(n^3)$ : this is really bad. If you increase the number of data items two times, the execution time grows 8 times. A typical example of an operation with this complexity is matrix multiplication. This is why you should always do whatever you can to avoid matrix multiplication in the code you write if you want to get a decent performance. Again, sometimes you can not do this, but usually it is worth trying.
7. $O(2^n)$ : this is one of the most extreme cases. Code of this complexity is virtually useless for any non-trivial task. We have seen an example of a function with this complexity: the recursive implementation of the `fibonnaci` function.

Now we will study complexity of some operations experimentally.

In [73]:
# Dict access by key is O(1) and list (and deque) access by index is O(n)
from collections import deque

L1   = list(range(10000))
D1   = deque(L1)
dct1 = { x: x for x in L1 }

L2   = list(range(100000))
D2   = deque(L2)
dct2 = { x: x for x in L2 }

idx = 5329

In [74]:
%%timeit
L1[idx]

41.6 ns ± 0.865 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [75]:
%%timeit
L2[idx]

39 ns ± 0.577 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [76]:
%%timeit
D1[idx]

122 ns ± 4.04 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [77]:
%%timeit
D2[idx]

153 ns ± 4.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [78]:
%%timeit
dct1[idx]

59.3 ns ± 2.63 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [79]:
%%timeit
dct2[idx]

60.8 ns ± 0.474 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


What can you conclude from the tests?

Now let us look at a problem of items by name and compare implementations based on lists and dicts.

In [80]:
L = [ ('a{}'.format(i), i) for i in range(100000) ]
dct = dict(L)

idx = 54225

In [81]:
%%timeit
next((name, value) for name, value in L if name == 'a{}'.format(idx))

13.6 ms ± 413 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [82]:
%%timeit
dct['a{}'.format(idx)]

272 ns ± 12.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [83]:
data = [('label1', 1), ('label2', 2)]

for element in data:
    if element[0] == 'label2':
        print(element)

('label2', 2)


In [84]:
dct = dict(data)
dct

{'label1': 1, 'label2': 2}

In [85]:
dct['label2']

2

## Homework

You have to design a new data management system for a university department. The application will process student data. Every student will be described with the following data fields:

* Name and surname (string, may not be unique)
* Student id (positive integer, unique)
* Gender ('F' or 'M')
* Study program (cognitive science, biology etc. | string)
* Year (1st, 2nd etc | integer)
* Date of birth (simplified just to year | integer)
* List of classes taken in the current semester (class ids are integers)

Program(s) and year(s) are provided in data records as 2-tuples
with program name and year at first and second positions.

The application has to meet the following requirements:

* Fast student lookup by name and surname
* Fast student lookup by student id
* Fast creation of a list of all students from a given year and given program
* Fast creation of a list of all students enrolled in a given class

Also the following constraints have to be accounted for:

* Some students can be enrolled at multiple programms at different years.
* Students' names and surnames may not be unique.

But remember that you can shape your data structure however you like,
rearrange data and copy it into multiple locations
(in this problem we do not care about synchronization of changes between different locations).
You may also implement as many helper functions as you want.

Also your solution should use the basic data structures covered in this class.
If you know more advanced stuff, then good for you, but in this problem you have
to work with the basics.

HINT. Avoid loops and prefer dictionary-based key lookups to get better performance.
      Remeber about requirements and constraints.

Skeleton code for functions is in `HW1.py` file. Test data is in `HW1_data.py` file
and is already imported in `HW1.py`.

Send your solution (`HW1.py`) to <stalaga@uw.edu.pl> or <jaroslaw.paszek@gmail.com> (depending on the group you are in) before 06.12.2019.

Use the following title in your e-mail: "Advanced Python HW1: [name and surname]"

Below is an example of how you can work on the problem within the notebook to test your implementations.

In [86]:
# Import HW1 objects.
# This will import both data: `records` and `records_many`
# as well as your functions (provided you implement them in `HW1.py`).
from HW1 import *

# Build your data structure, i.e.
data = build_data_structure(records)

In [87]:
%%time
# Test your function.
# `%%time` command in the code cell
# means that the execution time will be measured.
find_by_name(data, "some_name")

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [89]:
%%timeit
# `%%timeit` command runs the code in code cell many times
# in order to measure execution time with higher accuracy.
2 + 2

7.29 ns ± 0.0624 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)
